In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data_Washington Fatal Crash Survey.csv", low_memory=False)
zips = pd.read_csv("US%20Zip%20Codes%20from%202013%20Government%20Data.txt")

In [9]:
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS



#'methdrug', 'drugsts', drugtst1-12, drugres1-12, 'alcres', 'alcsts', 'alctst'
#ejectpath alcmeth
#investjur repjur

We will drop all columns that are more than 90% NANs because there are not enough values for them to be significant. We'll also drop columns that are not relevant to the problem statement: ['hosp_tm','arr_tm','not_tm','investjur','repjur','lab','methdrug','alcmeth','drugsts','drugtst1']

## Derive zip code for crashes 

Method: Reverse geocoding with ArcGIS and impute any missing zipcode values

In [10]:
# long : x
# lat : y

# driver zip code : dzip 
df.rename(columns={'x': 'lon', 'y': 'lat'}, inplace=True)

In [12]:
gis = GIS(api_key="AAPK206d8c337d5a416ca8a7824330c0ca7bkQWmIxSD8K5_qxX1ly5btaWLcl36cVX_1iDwCQxl_IgY56Zh6TN-Usx26i4oM77l")
# exposed for now

def get_zip(df,lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]),"y":float(df[lat_field]), "spatialReference":{"wkid":4326}})))
    return location['address']['Postal']

df['crashzip'] = df.apply(get_zip, axis=1, lat_field='lat', lon_field='lon')

In [83]:
# update missing crash zip values in df 
# case 78 03/18/2020 in county 45(Pacific) --> crash lat long improperly recorded  78.304936,  -778.304936 impossible! city = unincorporated
# FOREST SERVICES RD 2300 is not an actual road... should we drop the row?
# case 349 09/12/2021 in county 5 --> zip: 99352 could not be identified by arcGIS identified using haversine distance
# case 638 09/05/2021 in county 19(Ferry) --> lat long recorded as 78.304936,  -778.304936 impossible! city = unincorporated
# Ferry county is on a reservation in the middle of nowhere next to a natl forest far northeast of WA 
# crash occurred on BRIDGE CREEK RD in Ferry County --> brute force zip: 99138

df.loc[3690,"crashzip"] = 99352
df.loc[4128,"crashzip"] = 99138

In [84]:
# saved because computationally expensive 
# don't want to call API every time we run the code 
# DONT RUN THAT PART 

df.to_csv('crashdata_updated.csv') # import THIS data set

## What proportion of drivers involved in fatal crashes crash in communities where they live?

Method: Calculate proportion mathematically

In [384]:
# start from here

data = pd.read_csv("crashdata_updated.csv", low_memory=False) # some cols have mixed dtypes...
data["crash_dt"]= pd.to_datetime(data["crash_dt"])
data["licstate"]= data["licstate"].astype(str)

# Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

# How do we want to define "community"? Zip code should be fine... 
data['dzip'] = data['dzip'].astype(str)
# get rid of the trailing zeros
data['dzip'] = data['dzip'].replace(r'\.0$', '', regex=True)

In [595]:
data

,index,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,...,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,crashzip,d_res,dcity,crashcity,d_cty
0,0,0,2017,1,E628946,2.0,2017-01-01,2:12,1,1,...,NaN,NaN,NaN,NaN,NaN,98201,visitor,Everett,Everett,resident
1,1,1,2017,2,E627989,26.0,2017-01-02,17:14,2,1,...,NaN,NaN,NaN,NaN,NaN,98359,visitor,Tacoma,Olalla,visitor
2,2,2,2017,4,3747633,263.0,2017-01-01,18:47,1,1,...,NaN,NaN,NaN,NaN,NaN,98103,visitor,Seattle,Seattle,resident
3,3,3,2017,5,E628691,4.0,2017-01-01,3:50,1,1,...,NaN,NaN,NaN,NaN,NaN,99224,visitor,Mead,Spokane,visitor
4,4,4,2017,6,3746306,263.0,2017-01-05,9:53,5,1,...,NaN,NaN,NaN,NaN,NaN,98104,visitor,Snoqualmie,Seattle,visitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,4127,4127,2021,636,Incid08,334.0,2021-08-08,9:38,8,8,...,NaN,NaN,NaN,NaN,NaN,99116,visitor,Nespelem,Coulee dam,visitor
4027,4128,4128,2021,638,Incid09,334.0,2021-09-05,1:36,5,9,...,NaN,NaN,NaN,NaN,NaN,99138,resident,Inchelium,Inchelium,resident
4028,4129,4129,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,NaN,NaN,NaN,NaN,NaN,98144,visitor,Lacey,Seattle,visitor
4029,4130,4130,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,NaN,NaN,NaN,NaN,NaN,98144,resident,Seattle,Seattle,resident


In [385]:
# In order for this calculation to be accurate, we need to remove the rows where BOTH zip code and state is "unknown" or "not reported" because the driver zip for each of these rows won't match the crash zip but we have no way validating where the driver is ACTUALLY from-- we cannot assume that the driver is from out of state/country 

# Answers hinge on the assumption that the driver's zip code for each observation is accurate 
for index, row in data.iterrows():
    if row['dzip']=='99999' and row['licstate']=='99':
        data.drop(index, inplace=True)
    if row['dzip']=='99999' and row['licstate']=='98':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='99':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='98':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='53':
        data.drop(index, inplace=True)

In [386]:
# should be done

data.reset_index(inplace=True)

In [387]:
# For the crashes where driver's zip is "unknown" (99999) but state is known --> ok to keep since they are out of state
print(data[['licstate','dzip']].iloc[1359], "State/Country=Alabama") # index 1359 licstate = 1 (ALABAMA) 
print(data[['licstate','dzip']].iloc[984], "State/Country = Canada") # index 984 licstate = 95 (CANADA) No ZIP 

licstate        1
dzip        99999
Name: 1359, dtype: object State/Country=Alabama
licstate       95
dzip        99999
Name: 984, dtype: object State/Country = Canada


In [538]:
print("The proportion of drivers involved in fatal crashes where they don't live = {}".format(len(data.query('dzip != crashzip'))/len(data)))
print("The proportion of drivers involved in fatal crashes where they live = {}".format(len(data.query('dzip == crashzip'))/len(data)))

The proportion of drivers involved in fatal crashes where they don't live = 0.7568841478541305
The proportion of drivers involved in fatal crashes where they live = 0.2431158521458695


- Calculate the same proportions for before Spring 2020 and after Spring 2020
- Represent these 3 proportions in a horizontal stacked bar chart 

## Recalculate proportions-- this time community = CITY

In [532]:
# Source: https://github.com/scpike/us-state-county-zip
citydata = pd.read_csv("geo-data.csv")

In [534]:
# match city to zip codes

def matchcity(df, dct):
    dct = dict(zip(citydata['zipcode'], citydata['city']))
    df['dcity']=df['dzip'].map(dct).fillna('00') # map over df 
    df['crashcity']=df['crashzip'].map(dct).fillna('00') # map over df 
    return df

In [535]:
citydict = dict(zip(citydata['zipcode'], citydata['city']))

# call our function 
matchcity(data, citydict)

,index,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,...,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,crashzip,d_res,dcity,crashcity
0,0,0,2017,1,E628946,2.0,2017-01-01,2:12,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,98201,visitor,Everett,Everett
1,1,1,2017,2,E627989,26.0,2017-01-02,17:14,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,98359,visitor,Tacoma,Olalla
2,2,2,2017,4,3747633,263.0,2017-01-01,18:47,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,98103,visitor,Seattle,Seattle
3,3,3,2017,5,E628691,4.0,2017-01-01,3:50,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,99224,visitor,Mead,Spokane
4,4,4,2017,6,3746306,263.0,2017-01-05,9:53,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,98104,visitor,Snoqualmie,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,4127,4127,2021,636,Incid08,334.0,2021-08-08,9:38,8,8,...,NaN,NaN,NaN,NaN,NaN,NaN,99116,visitor,Nespelem,Coulee dam
4027,4128,4128,2021,638,Incid09,334.0,2021-09-05,1:36,5,9,...,NaN,NaN,NaN,NaN,NaN,NaN,99138,resident,Inchelium,Inchelium
4028,4129,4129,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,NaN,NaN,NaN,NaN,NaN,NaN,98144,visitor,Lacey,Seattle
4029,4130,4130,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,NaN,NaN,NaN,NaN,NaN,NaN,98144,resident,Seattle,Seattle


In [537]:
print("The proportion of drivers involved in fatal crashes in communities where they don't live = {}".format(len(data.query('dcity != crashcity'))/len(data)))
print("The proportion of drivers involved in fatal crashes in communities where they live = {}".format(len(data.query('dcity == crashcity'))/len(data)))

The proportion of drivers involved in fatal crashes in communities where they don't live = 0.6889109402133465
The proportion of drivers involved in fatal crashes in communities where they live = 0.3110890597866534


- Calculate the same proportions for before Spring 2020 and after Spring 2020
- Represent these 3 proportions in a horizontal stacked bar chart 

## Is there a difference between types of crashes and behavior factors among residents vs visitors?

Method: Chi-Squared Test for Homogeneity between crashes among residents and visitors

- null hypothesis: Visitor and Resident populations are homogeneous regarding the proportions of categories of categorical variables (crash/behavior factors)

In [391]:
# Create a categorical variable to represent whether driver is local resident or visitor
# "resident" = local resident
# "visitor" = out of town

drive_res = [] # set empty list 
for index, row in data.iterrows():
    if row['dzip']==row['crashzip']:
        drive_res.append("resident") # if equal add "resident" to list
    else:
        drive_res.append("visitor") # else add "visitor"
        
data["d_res"] = drive_res # cast list as df column

In [542]:
# Create a categorical variable to represent whether driver is local resident or visitor
# "resident" = local resident
# "visitor" = out of town

### BASED ON CITY MATCH

city_res = [] # set empty list 
for index, row in data.iterrows():
    if row['dcity']==row['crashcity']:
        city_res.append("resident") # if equal add "resident" to list
    else:
        city_res.append("visitor") # else add "visitor"
        
data["d_cty"] = city_res # cast list as df column

# "d_cty" : classifies driver as resident or visitor by CITY 
# "d_res" : classifies driver as resident or visitor by ZIP

In [203]:
# method: chi-squared test of homogeneity 

# Is there a difference between the types of crashes among residents vs visitors?
# Is there a difference between the behavior factors among residents vs visitors?

In [392]:
# experimental crosstabs to look at the data lol

# crash level
crash_tab = pd.crosstab([data.numfatal,data.roadclass,data.pv_inv,data.ped_inv,data.ht_inv,data.bike_inv,data.mc_inv],data.d_res, rownames = ["Fatalities", "RoadClass","PV_inv","PED_inv","HT_inv","Bike_inv", "MC_inv"],colnames = ["Driver"],margins=True, margins_name="Total")

crashtype = pd.crosstab([data.numfatal,data.crashtype],data.d_res, rownames = ["Fatalities", "Crashtype"],colnames = ["Driver"],margins=True, margins_name="Total")

# driver behavior level
behav_tab = pd.crosstab([data.dd_inv,data.speed_inv,data.drowsy_inv,data.drdist_inv],data.d_res, rownames = ["Drinking","Speeding","Drowsy","Distract"],colnames = ["Driver"],margins=True, margins_name="Total")

In [393]:
crashtype

Driver                resident  visitor  Total
Fatalities Crashtype                          
1          0                 4       15     19
           1                89      233    322
           2                34       91    125
           3                 1        1      2
           5                 1        1      2
...                        ...      ...    ...
4          50                0        1      1
           51                1        0      1
           86                0        1      1
           87                0        1      1
Total                      980     3051   4031

[115 rows x 3 columns]

In [394]:
behav_tab

Driver                             resident  visitor  Total
Drinking Speeding Drowsy Distract                          
0        0        0      0              355     1120   1475
                         1              149      406    555
                  1      0               15       35     50
                         1                0        3      3
         1        0      0              124      490    614
                         1               15       64     79
                  1      0                2        4      6
1        0        0      0              126      452    578
                         1               46      101    147
                  1      0                2        6      8
         1        0      0              137      328    465
                         1                9       42     51
Total                                   980     3051   4031

## Are there specific resident ZIP Codes that tend to produce higher-risk drivers that are involved in fatal crashes at a higher rate?

- Keywords: produce... tend... and rate...

- For simplicity we'll just assume that "high risk" = all of these drivers since they all killed at least 1 person 
- Calculate: # of high risk drivers/total population of zip code

In [380]:
# note: we know there are 28 fatal crashes caused by drivers from CANADA

In [577]:
temp = pd.DataFrame(data.groupby(data.crash_dt.dt.year)['dzip'].value_counts())

In [579]:
temp.rename(columns={'dzip':'counts'}, inplace=True)
temp.reset_index(inplace=True)

In [581]:
temp

,crash_dt,dzip,counts
0,2017,98258,11
1,2017,98223,9
2,2017,99301,9
3,2017,0,8
4,2017,98002,8
...,...,...,...
1731,2021,99371,1
1732,2021,99508,1
1733,2021,99692,1
1734,2021,99998,1


In [590]:
temp.loc[temp['dzip']=='99301']

,crash_dt,dzip,counts
2,2017,99301,9
376,2018,99301,5
684,2019,99301,9
1038,2020,99301,12
1378,2021,99301,12


In [594]:
# rate of drivers causing fatal crashes per zipcode per year
data['dzip'].value_counts()/5

99301    9.4
98444    8.6
98387    7.0
98837    6.4
98404    6.4
        ... 
33478    0.2
95377    0.2
72712    0.2
98035    0.2
85602    0.2
Name: dzip, Length: 701, dtype: float64

In [395]:
zipfreq = pd.DataFrame(data['dzip'].value_counts()).reset_index()

In [548]:
popzip = pd.read_csv("population_by_zip_2010.csv")
popzip['zipcode'] = popzip['zipcode'].astype(str)
popzip['zipcode'] = popzip['zipcode'].replace(r'\.0$', '', regex=True)

In [453]:
zipdemo = pd.read_csv("washzips_u.csv")
zipdemo.rename(columns={'Zip Code':'zip','Population':'pop'}, inplace=True)
zipdemo = zipdemo.loc[0:1294]
zipdemo['zip'] = zipdemo['zip'].astype(str)
zipdemo['zip'] = zipdemo['zip'].replace(r'\.0$', '', regex=True)
zipdemo['pop'] = zipdemo['pop'].replace(',', '', regex=True).astype(int)

In [551]:
# make a copy

populationzip = zipfreq.copy()

In [555]:
# new population matches 

def new_match(df, maps):
    maps = dict(zip(popzip['zipcode'], popzip['population']))
    df['population']=df['index'].map(maps).fillna('00') # map over df 
    return df

In [556]:
# new pop matches
maps = dict(zip(popzip['zipcode'], popzip['population']))

new_match(populationzip, maps)

,index,dzip,pop,population
0,99301,47,84124.0,2788.0
1,98444,43,35083.0,1056.0
2,98387,35,52017.0,240.0
3,98837,32,45729.0,851.0
4,98404,32,35357.0,1301.0
...,...,...,...,...
696,33478,1,00,151.0
697,95377,1,00,115.0
698,72712,1,00,1520.0
699,98035,1,00,00


In [455]:
def get_match(df, dct):
    dct = dict(zip(zipdemo['zip'], zipdemo['pop']))
    df['pop']=df['index'].map(dct).fillna('00') # map over df 
    return df

In [456]:
# create dictionary
dct = dict(zip(zipdemo['zip'], zipdemo['pop']))

# call function to append population
get_match(zipfreq, dct)

,index,dzip,pop
0,99301,47,84124.0
1,98444,43,35083.0
2,98387,35,52017.0
3,98837,32,45729.0
4,98404,32,35357.0
...,...,...,...
696,33478,1,00
697,95377,1,00
698,72712,1,00
699,98035,1,00


In [485]:
# drop observations = 1 focus on 'high risk' populations

zipcount = zipfreq[zipfreq['dzip'] > 1]

In [486]:
# 98507 : 3
# 98508 : 2
# 0/99999 : 28
# we'll drop those cols for clarity in this answer because we cannot approximate the population of Canada 

# brute force impute the rest 
zipcount.iloc[310, zipcount.columns.get_loc('pop')] = 516
zipcount.iloc[344, zipcount.columns.get_loc('pop')] = 71118
zipcount.iloc[356, zipcount.columns.get_loc('pop')] = 39926
zipcount.iloc[358, zipcount.columns.get_loc('pop')] = 68865
zipcount.iloc[381, zipcount.columns.get_loc('pop')] = 9856

In [494]:
# This is to avoid setting with copy warning 

z = zipcount.copy()
# rename this column
z.rename(columns={'index':'zip'}, inplace=True)
z['population'] = z['pop'].astype(int)
zipct = z[z['population'] >0]

In [504]:
zipct.reset_index(inplace=True)

In [509]:
# Calculate proprotion of drivers/zip population size
riskrate = []

for index, row in zipct.iterrows():
    riskrate.append(row['dzip']/(row['population']))
    
zipct["riskrate"] = riskrate

/var/folders/j2/8gptfrvd0_dfl_yxfcck_hh40000gn/T/ipykernel_25414/2591585461.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipct["riskrate"] = riskrate


In [518]:
zipct.sort_values(by='riskrate',ascending=False)

,index,zip,dzip,pop,population,riskrate
273,274,98539,4,136.0,136,0.029412
367,370,98832,2,155.0,155,0.012903
349,352,98582,2,177.0,177,0.011299
316,318,99371,3,314.0,314,0.009554
351,354,98552,2,240.0,240,0.008333
...,...,...,...,...,...,...
353,356,99508,2,39926,39926,0.000050
356,359,97756,2,42180.0,42180,0.000047
374,377,97124,2,54287.0,54287,0.000037
355,358,84015,2,68865,68865,0.000029
